In [22]:
from scipy.stats import norm
import gurobipy as gp
from gurobipy import GRB
import pandas as pd 
import numpy as np
import time
import math

"""
first compute composition in new way, then transfer this composition to the 200 cross-sections
afterwards check which percentage it will be overcrowded and compute weighted indicator
maybe compute multiple indicators with different weighting formulas
then compare the results and the costs of what you found

compute costs for different satisfaction levels for extension and basic problem
"""

class OPTIMISER:
    
    def __init__(self):
        pass

    def iteration(self):
        ###################################################################################################
        # variable parameters to indicate amount of optimizations
        dimDemand = 1
        dimCapacity = 5
        dimSatisfaction = 5
        ###################################################################################################

        total = dimDemand * dimCapacity * dimSatisfaction
        results = np.empty((total, 5))
        d = 0

        for a in range(dimDemand):
            for b in range(dimSatisfaction):
                for c in range(dimCapacity):
                    print(str(a) + " " + str(b) + " " + str(c) + " " + str(d))
                    optimizer.example(results, a, b, c, d)
                    d = d + 1

        print(results)
        df = pd.DataFrame(results, columns=['demand', 'satisfaction', 'capacity', 'costs', 'time'])
        df.to_excel('extension1_results.xlsx', sheet_name='Results')
    
    def example(self, results, dem, sat, cap, counter):
        try:
            ###################################################################################################
            # variable parameters for step sizes and basis levels
            demandFirst = 0.1 + 0.01 * dem
            capacityFirst = 0.1 + 0.02 * cap
            satisfyFirst = 0.8 + 0.03 * sat
            timeLimit = 3
            ###################################################################################################            

            results[counter][0] = demandFirst
            results[counter][2] = capacityFirst
            results[counter][1] = satisfyFirst

            m = gp.Model("mip1")

            # Import data 
            path = "C:\\Users\\Arbeitsaccount\\Downloads\\SeminarCase1\\length_param.xlsx"
            df1 = pd.read_excel(path)
            path = "C:\\Users\\Arbeitsaccount\\Downloads\\SeminarCase1\\data_compos.xlsx"
            df2 = pd.read_excel(path)
            path = "C:\\Users\\Arbeitsaccount\\Downloads\\SeminarCase1\\Railway services-2024.xlsx"
            df3 = pd.read_excel(path)
            #df3 = df3.iloc[:-3]
            f = df2["Costs"]
            set = ["OH", "OC"]
  
            dim1 = 200
            dim2 = 10

            # split up demand into 1st and 2nd class
            demand1 = []
            demand2 = []
            deviation1 = []
            deviation2 = []
            for c in range(200):
                demand1.append(demandFirst * df3["Demand(μ)"][c])
                demand2.append((1 - demandFirst) * df3["Demand(μ)"][c])
                deviation1.append(math.sqrt(demandFirst * math.pow(df3["Demand(σ)"][c], 2)))
                deviation2.append(math.sqrt((1 - demandFirst) * math.pow(df3["Demand(σ)"][c], 2)))
            #print(demand1)
            #print(demand2)
            #print(deviation1)
            #print(deviation2) 
                        
            #probability array
            prob = np.empty((dim1, dim2))
            prob1 = np.empty((dim1, dim2))
            prob2 = np.empty((dim1, dim2))
            for c in range(200):
                for p in range(10):
                    prob[c,p] = norm.cdf(round(1 * (df2["Capacity"][p] - 1), 0), loc=df3["Demand(μ)"][c], scale=df3["Demand(σ)"][c])
                    prob1[c,p] = norm.cdf(round(capacityFirst * (df2["Capacity"][p] - 1), 0), loc=demand1[c], scale=deviation1[c])
                    prob2[c,p] = norm.cdf(round((1 - capacityFirst) * (df2["Capacity"][p] - 1), 0), loc=demand2[c], scale=deviation2[c])
                    #print(str(prob[c,p]) + " " + str(prob1[c,p]) + " " + str(prob2[c,p]))
    	    
            """
            # compute probability array for first class
            prob1 = np.empty((dim1, dim2))
            for c in range(200):
                for p in range(10):
                    prob1[c,p] = norm.cdf(round(capacityFirst * (df2["Capacity"][p] - 1), 0) , loc=demand1[c], scale=deviation1[c])

            # compute probability array for second class
            prob2 = np.empty((dim1, dim2))
            for c in range(200):
                for p in range(10):
                    prob2[c,p] = norm.cdf(round((1 - capacityFirst) * (df2["Capacity"][p] - 1), 0) , loc=demand2[c], scale=deviation2[c])
            """
            # record the start time   
            start_time = time.time()  
            
            x = {}
            for c in range(200):  # Define the first index range
                for p in range(10) :  # Define the second index range
                    x[c, p] = m.addVar(vtype=GRB.BINARY, name=f"x[{c},{p}]") 
            
            # objective function
            m.setObjective(gp.quicksum(x[c, p] * f[p] for c in range(200) for p in range(10)), GRB.MINIMIZE)
            
            # constraint 1: only one composition
            for c in range(200):
                m.addConstr(gp.quicksum(x[c, p] for p in range(10)) == 1)
                #m.addConstr(gp.quicksum(x[c, p] * df1.iloc[c, p] for p in range(10)) == 1)
                #if df3["Line"][c] == 400:
                    #m.addConstr(gp.quicksum(x[c, p] * df2["Length"][p] for p in range(10)) <= 200)

            # constraint 2: composition needs to fit the platform length
            for c in range(200):
                m.addConstr(gp.quicksum(x[c, p] * df1.iloc[c, p] for p in range(10)) == 1)

            # constraint 3: difference between rolling stock types
            for u in set:
                for k in set:
                    if u != k:
                       m.addConstr(gp.quicksum(x[c,p] * df2[u][p] - x[c,p] * df2[k][p]  for c in range(200) for p in range(10)) <= gp.quicksum(x[c,p] * df2[k][p]  for c in range(200) for p in range(10)) * 0.25)

            # constraint 4: total seating requirement 1st class
            m.addConstr(gp.quicksum(x[c,p] * prob1[c,p] for c in range(200) for p in range(10)) >= satisfyFirst * 200)

            # constraint 5: total seating requirement 2nd class
            m.addConstr(gp.quicksum(x[c,p] * prob2[c,p] for c in range(200) for p in range(10)) >= 0.81 * 200)

            # constraint 6: seating requirements per cross-section 1st class
            for c in range(200):
                m.addConstr(gp.quicksum(x[c,p] * round(capacityFirst * (df2["Capacity"][p] - 1), 0) for p in range(10)) >= demand1[c])

            # constraint 7: seating requirements per cross-section 2st class
            for c in range(200):
                m.addConstr(gp.quicksum(x[c,p] * round((1 - capacityFirst) * (df2["Capacity"][p] - 1), 0) for p in range(10)) >= demand2[c])


            m.setParam('TimeLimit', timeLimit)
            m.optimize()
            
            #for v in m.getVars():
                #print("%s %g" % (v.varName, v.x))
            
            print("Obj: %g" % m.objVal)
            elapsed_time = time.time() - start_time  # Calculate elapsed time
            print("Optimization runtime:", elapsed_time, "seconds")

            
            results[counter][3] = m.objVal
            results[counter][4] = elapsed_time
            
        except gp.GurobiError as e:
            print("Error code " + str(e.errno) + ": " + str(e))
        except AttributeError:
            print("Encountered an attribute error")


if __name__ == "__main__":
    optimizer = OPTIMISER()
    optimizer.iteration()


0 0 0 0
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter TimeLimit to value 3
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-7200U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 804 rows, 2000 columns and 15833 nonzeros
Model fingerprint: 0x953cf741
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [2e-13, 2e+03]
  Objective range  [2e+05, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.223600e+08
Presolve removed 604 rows and 530 columns
Presolve time: 0.05s
Presolved: 200 rows, 1470 columns, 6645 nonzeros
Variable types: 0 continuous, 1470 integer (1470 binar